# 1. 드라이브 연결 및 환경 설정

- point: 구글드라이브 마운트, 파일 위치/경로 확인

In [ ]:
# 개인의 드라이브 계정에 연결 (마운트)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 한국어 형태소 분석에 필요한 konlpy 설치
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.6/465.6 KB 50.5 MB/s eta 0:00:00


In [ ]:
# 필요한 라이브러리를 import
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import json
import re
import zipfile

import os

# 2. 압축 해제 및 train/validation 데이터 추출, 데이터프레임으로 저장

- point: 제시된 train/test 데이터를 응용하여 알맞은 train/test 비율로 조정 가능 (concat 등 함수 이용)

#### 압축 해제

In [ ]:
# 훈련 데이터를 가져오기 위해 압축 파일을 해제
with zipfile.ZipFile('/content/drive/MyDrive/hack/143.민원 업무 효율, 자동화를 위한 언어 AI 학습데이터/143.민원 업무 효율, 자동화를 위한 언어 AI 학습데이터/01.데이터/1.Training/라벨링데이터/TL1.zip', 'r') as zip_file:
  for file in zip_file.infolist():
    file.filename = file.filename.encode('cp437').decode('utf-8')
    zip_file.extract(file, '/content/drive/MyDrive/hack/143.민원 업무 효율, 자동화를 위한 언어 AI 학습데이터/143.민원 업무 효율, 자동화를 위한 언어 AI 학습데이터/01.데이터/1.Training/라벨링데이터')

In [ ]:
# 테스트 데이터를 가져오기 위해 압축 파일을 해제
with zipfile.ZipFile('/content/drive/MyDrive/hack/143.민원 업무 효율, 자동화를 위한 언어 AI 학습데이터/143.민원 업무 효율, 자동화를 위한 언어 AI 학습데이터/01.데이터/2.Validation/라벨링데이터/VL1.zip', 'r') as zip_file:
  for file in zip_file.infolist():
    file.filename = file.filename.encode('cp437').decode('utf-8')
    zip_file.extract(file, '/content/drive/MyDrive/hack/143.민원 업무 효율, 자동화를 위한 언어 AI 학습데이터/143.민원 업무 효율, 자동화를 위한 언어 AI 학습데이터/01.데이터/2.Validation/라벨링데이터')

#### 데이터 추출

In [ ]:
# 분류하고자 하는 민원 종류에 대한 딕셔너리 변수 만들기
# context: ['안전건설', '교통', '건축허가', '환경미화'] 를 숫자형태로 변경
label_di = {'안전건설':0, '교통':1, '건축허가':2, '환경미화':3}

In [ ]:
subject_li = list(label_di.keys())

total_train_li = []
total_test_li = []

# training data
for subject in subject_li:
  subject_dir = os.listdir(f'/content/drive/MyDrive/hack/143.민원 업무 효율, 자동화를 위한 언어 AI 학습데이터/143.민원 업무 효율, 자동화를 위한 언어 AI 학습데이터/01.데이터/1.Training/라벨링데이터/{subject}/')[0]
  with open(f'/content/drive/MyDrive/hack/143.민원 업무 효율, 자동화를 위한 언어 AI 학습데이터/143.민원 업무 효율, 자동화를 위한 언어 AI 학습데이터/01.데이터/1.Training/라벨링데이터/{subject}/{subject_dir}') as f:
    temp_json = json.load(f)
    text_li = [(data["Q_refined"], label_di[subject]) for data in temp_json['documents']]
    total_train_li.extend(text_li)

# test data
for subject in subject_li:
  subject_dir = os.listdir(f'/content/drive/MyDrive/hack/143.민원 업무 효율, 자동화를 위한 언어 AI 학습데이터/143.민원 업무 효율, 자동화를 위한 언어 AI 학습데이터/01.데이터/2.Validation/라벨링데이터/{subject}/')[0]
  with open(f'/content/drive/MyDrive/hack/143.민원 업무 효율, 자동화를 위한 언어 AI 학습데이터/143.민원 업무 효율, 자동화를 위한 언어 AI 학습데이터/01.데이터/2.Validation/라벨링데이터/{subject}/{subject_dir}') as f:
    temp_json = json.load(f)
    text_li = [(data["Q_refined"], label_di[subject]) for data in temp_json['documents']]
    total_test_li.extend(text_li)

In [ ]:
# 데이터 개수 출력
print(len(total_train_li),len(total_test_li))

274597 34325


In [ ]:
# 자료 형태 확인
total_train_li[0]

('의창구 #@주소# 놀이기구가 많이 부식되어서 조치가 필요해 보입니다. 확인 후 처리 바랍니다.', 0)

In [ ]:
type(total_train_li) 

list

#### 데이터프레임으로 저장

In [ ]:
# 훈련데이터와 테스트 데이터를 데이터프레임화 시킴
train_df = pd.DataFrame(total_train_li, columns=['contents','class'])
test_df = pd.DataFrame(total_test_li, columns=['contents','class'])

#### 데이터프레임 확인

In [ ]:
# 훈련데이터 출력
train_df

,contents,class
0,의창구 #@주소# 놀이기구가 많이 부식되어서 조치가 필요해 보입니다. 확인 후 처리...,0
1,의창구 #@주소# 놀이기구 부식 상태가 심하네요. 관리가 필요해 보입니다.,0
2,부식 상태가 심각한 놀이기구가 있으니 조치 바랍니다. 위치는 의창구 #@주소#입니다.,0
3,"의창구 #@주소# 인근 놀이터 입구 쇠 막대 돌출, 철제 돌출에 대하여 제거 요청합니다.",0
4,쇠 막대와 철제 돌출에 대하여 제거 부탁드립니다. 위치는 의창구 #@주소# 인근 놀...,0
...,...,...
274592,창원시청 노후 보일러 교체 지원금의 확인은 어느 부서로 할까요?,3
274593,지금 창원시청 노후 보일러 교체시 지원금을 확인하려고 합니다. 부서 확인 부탁드립니다.,3
274594,진해시에 있는 직원을 하나 찾고싶어요. 50대 후반 여성이고 환경과에 근무하고 있어요.,3
274595,환경과에 50대 후반의 황#@이름#라는 직원이 근무하고 있습니까?,3


In [ ]:
# 테스트데이터 출력
test_df

,contents,class
0,놀이기구 부식이 심해 제보합니다. 위치는 의창구 #@주소#이고 조치 좀 해주세요.,0
1,어린이 학원 앞의 도로가 패여 있습니다. 조속한 보수를 해주시기 바랍니다.,0
2,창원시에서 평탄하지 못한 인도로 인하여 보행자 안전사고의 위험이 있습니다. 현장점검...,0
3,차가 다니는 골목길에 주차금지 장애물이 합법인지 문의합니다.,0
4,아파트 뒤편 도로 보수 공사를 요청합니다. 자칫하다가는 큰 사고로 이어질 수 있으니...,0
...,...,...
34320,진해구 돌아다니고 있는 야생멧돼지를 발견하고 신고 문의 드립니다.,3
34321,의창구 #@주소#에 있는 뉴트리아들 포획 부탁드려요.,3
34322,"문의합니다. 마산합포구 #@주소#의 화장실이 개방형 #@주소#,1, 2층 남녀 화장...",3
34323,성산구의 생활폐기물매립장 연락처 문의합니다.,3


# 3. 데이터 전처리

- point: stopwords, preprocessing 커스터마이징
- point: 전처리 과정이 너무 긴 경우, pickle 형식으로 저장/불러오기 사용

#### preprocessing

In [ ]:
# 한국어 형태소 분석기 Okt를 사용하기 위해 import
from konlpy.tag import Okt

In [ ]:
# Okt의 객체 생성
okt = Okt()

# 한국어에서 의미가 거의 없는 단어들을 불용어 처리하기 위해 집합을 만듦 - stopwords
stop_words = set(['은', '는', '이', '가', '아', '하', '들', '것', '의', '있', '되', '수', '보', '주', '등', '한', '에'])

In [ ]:
# 전처리 함수
def preprocessing(okt, review):
  # 한글이 아닌 단어는 지우고 " "로 대체
  review_text = re.sub('[^ ㄱ-ㅣ가-힣 0-9]+'," ",str(review))
  # 형태소분석을 하며 stemming 기법을 사용
  token_li = okt.morphs(review_text, stem = True)
  # 이전 stop_words에 포함되지 않는 단어만 clean_review 에 들어갈 수 있게 함
  clean_review = [token for token in token_li if token not in stop_words]
  # 단어들을 ' '기준으로 이어 붙혀서 하나의 string 타입 변수를 생성함
  clean_review =  " ".join(clean_review)

  # 전처리 완료된 문장을 return
  return clean_review.strip()

In [ ]:
from tqdm import tqdm
tqdm.pandas()

# preprocessing 함수를 적용 (.progress_apply 함수 활용 가능)
train_df['prepro_data'] = train_df['contents'].progress_apply(lambda x: preprocessing(okt, x))
test_df['prepro_data'] = test_df['contents'].progress_apply(lambda x: preprocessing(okt, x))

100%|██████████| 34325/34325 [01:49<00:00, 313.84it/s]


In [ ]:
# 전처리된 데이터를 데이터프레임의 prepro_data 열에서 확인
train_df

,contents,class,prepro_data
0,의창구 #@주소# 놀이기구가 많이 부식되어서 조치가 필요해 보입니다. 확인 후 처리...,0,의창구 주소 놀이기구 많이 부식 되어다 조치 필요하다 보이다 확인 후 처리 바라다
1,의창구 #@주소# 놀이기구 부식 상태가 심하네요. 관리가 필요해 보입니다.,0,의창구 주소 놀이기구 부식 상태 심하다 관리 필요하다 보이다
2,부식 상태가 심각한 놀이기구가 있으니 조치 바랍니다. 위치는 의창구 #@주소#입니다.,0,부식 상태 심각하다 놀이기구 있다 조치 바라다 위치 의창구 주소 이다
3,"의창구 #@주소# 인근 놀이터 입구 쇠 막대 돌출, 철제 돌출에 대하여 제거 요청합니다.",0,의창구 주소 인근 놀이터 입구 쇠 막대 돌출 철제 돌출 대하 여 제거 요청 하다
4,쇠 막대와 철제 돌출에 대하여 제거 부탁드립니다. 위치는 의창구 #@주소# 인근 놀...,0,쇠 막대 와 철제 돌출 대하 여 제거 부탁드리다 위치 의창구 주소 인근 놀이터 입구 이다
...,...,...,...
274592,창원시청 노후 보일러 교체 지원금의 확인은 어느 부서로 할까요?,3,창원 시청 노후 보일러 교체 지 원금 확인 어느 부서 로 하다 요
274593,지금 창원시청 노후 보일러 교체시 지원금을 확인하려고 합니다. 부서 확인 부탁드립니다.,3,지금 창원 시청 노후 보일러 교체 시 지 원금 을 확인 하다 하다 부서 확인 부탁드리다
274594,진해시에 있는 직원을 하나 찾고싶어요. 50대 후반 여성이고 환경과에 근무하고 있어요.,3,진해시 있다 직원 을 하나 찾다 50 대다 후반 여성 이고 환경 과 근무 하고 있다
274595,환경과에 50대 후반의 황#@이름#라는 직원이 근무하고 있습니까?,3,환경 과 50 대다 후반 황 이름 라는 직원 근무 하고 있다


In [ ]:
# 전처리된 데이터를 데이터프레임의 prepro_data 열에서 확인
test_df

,contents,class,prepro_data
0,놀이기구 부식이 심해 제보합니다. 위치는 의창구 #@주소#이고 조치 좀 해주세요.,0,놀이기구 부식 심해 제보 하다 위치 의창구 주소 이고 조치 좀 해주다
1,어린이 학원 앞의 도로가 패여 있습니다. 조속한 보수를 해주시기 바랍니다.,0,어린이 학원 앞 도로 패 여 있다 조속 보수 를 해주다 바라다
2,창원시에서 평탄하지 못한 인도로 인하여 보행자 안전사고의 위험이 있습니다. 현장점검...,0,창원시 에서 평탄하다 못 인도 로 인하다 보행자 안전 사고 위험 있다 현장 점검 부...
3,차가 다니는 골목길에 주차금지 장애물이 합법인지 문의합니다.,0,차갑다 다니다 골목길 주차 금지 장애물 합법 인지 문의 하다
4,아파트 뒤편 도로 보수 공사를 요청합니다. 자칫하다가는 큰 사고로 이어질 수 있으니...,0,아파트 뒤편 도로 보수 공사 를 요청 하다 자칫 하다 크다 사고 로 이어지다 있다 ...
...,...,...,...
34320,진해구 돌아다니고 있는 야생멧돼지를 발견하고 신고 문의 드립니다.,3,진해구 돌아다니다 있다 야생 멧돼지 를 발견 하고 신고 문의 드리다
34321,의창구 #@주소#에 있는 뉴트리아들 포획 부탁드려요.,3,의창구 주소 있다 뉴트리아 포획 부탁드리다
34322,"문의합니다. 마산합포구 #@주소#의 화장실이 개방형 #@주소#,1, 2층 남녀 화장...",3,문의 하다 마산합포구 주소 화장실 개방 형 주소 1 2 층 남녀 화장실 분리 개방 ...
34323,성산구의 생활폐기물매립장 연락처 문의합니다.,3,성산구 생활 폐기물 매립 장 연락처 문의 하다


In [ ]:
len(train_df)

274597

In [ ]:
# 전처리 과정에서 모든 문장이 삭제될 가능성이 있으므로, 해당 데이터 제거
  # train
print('중복 제거 전 훈련 데이터 개수: ', len(train_df))
train_df = train_df.drop_duplicates()
print('중복 제거 후 훈련 데이터 개수: ', len(train_df))

  # test
print('중복 제거 전 테스트 데이터 개수: ', len(test_df))
test_df = test_df.drop_duplicates()
print('중복 제거 후 테스트 데이터 개수: ', len(test_df))

중복 제거 전 훈련 데이터 개수:  274597
중복 제거 후 훈련 데이터 개수:  266615
중복 제거 전 테스트 데이터 개수:  34325
중복 제거 후 테스트 데이터 개수:  33748


In [ ]:
# 전처리 과정에서 모든 문장이 삭제될 가능성이 있으므로, 해당 데이터 제거
  # train
print('공백 제거 전 훈련 데이터 개수: ', len(train_df))
train_df = train_df[train_df['prepro_data'] != ""]
print('공백 제거 후 훈련 데이터 개수: ', len(train_df))

  # test
print('공백 제거 전 테스트 데이터 개수: ', len(test_df))
test_df = test_df[test_df['prepro_data'] != ""]
print('공백 제거 후 테스트 데이터 개수: ', len(test_df))

공백 제거 전 훈련 데이터 개수:  266615
공백 제거 후 훈련 데이터 개수:  266614
공백 제거 전 테스트 데이터 개수:  33748
공백 제거 후 테스트 데이터 개수:  33748


#### pickle

In [ ]:
# pickle 사용법
import pickle

with open('/content/drive/MyDrive/hack/pickle/train_df.pkl', 'wb') as f:
  pickle.dump(train_df, f) 
with open('/content/drive/MyDrive/hack/pickle/test_df.pkl', 'wb') as f:
  pickle.dump(test_df, f)

In [134]:
# 2.load
with open('/content/drive/MyDrive/hack/pickle/train_df.pkl', 'rb') as f:
  train_df = pickle.load(f)
with open('/content/drive/MyDrive/hack/pickle/test_df.pkl', 'rb') as f:
  test_df = pickle.load(f)

In [135]:
# 순서 변경
train_df = train_df.sample(frac = 1).reset_index(drop = True)
test_df = test_df.sample(frac = 1).reset_index(drop = True)

# 훈련 데이터
x_train = train_df['prepro_data'].copy()
y_train = train_df['class'].copy()

# 테스트 데이터
x_test = test_df['prepro_data'].copy()
y_test = test_df['class'].copy()

In [ ]:
# 데이터 사이즈 확인
print(len(train_df))
print(len(test_df))

266614
33748


# 4. 워드임베딩

- point: tf-idf vectorization 

In [ ]:
print(y_train.shape)# tfidf embedding
from sklearn.feature_extraction.text import TfidfVectorizer

# tfidf embedding 을 하기 위해, TfidfVectorizer의 객체를 선언
tfidf_vectorizer = TfidfVectorizer(ngram_range = (1,3))

# 훈련 데이터, 테스트 데이터 각각 transform 
x_tfidf_train = tfidf_vectorizer.fit_transform(x_train)
y_train = np.array(y_train)

x_tfidf_test = tfidf_vectorizer.transform(x_test)
y_test = np.array(y_test)

print(y_train, y_test)

(266614,)
[0 1 3 ... 0 1 0] [0 1 3 ... 0 0 0]


# 5. 모델 만들기 / 결과 확인

* GridSearch, RandomizedSearch 기법/manual기법 등을 사용해 hyperparameter 조정하는 과정이 반드시 필요
* 결과는 Accuracy Score 만으로 평가

- point: 모델/파라미터 선택 과정 설명

#### 예시: Decision Tree

- gridsearch 없이 진행

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt_tfidf_model = DecisionTreeClassifier()
dt_tfidf_model.fit(x_tfidf_train,y_train)

DecisionTreeClassifier()

In [ ]:
pred_y_tfidf = dt_tfidf_model.predict(x_tfidf_test)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,pred_y_tfidf))

              precision    recall  f1-score   support

           0       0.89      0.89      0.89     11704
           1       0.91      0.91      0.91     10175
           2       0.92      0.94      0.93      7132
           3       0.94      0.91      0.92      4737

    accuracy                           0.91     33748
   macro avg       0.91      0.91      0.91     33748
weighted avg       0.91      0.91      0.91     33748



In [ ]:
dt_tfidf_model.score(x_tfidf_train,y_train)

0.9989872999917484

In [ ]:
dt_tfidf_model.score(x_tfidf_test,y_test)

0.9078167595116747

In [ ]:
# DecisionTreeClassifier 모델 저장
import pickle

with open('/content/drive/MyDrive/hack/pickle/model.pkl', 'wb') as f:
  pickle.dump(dt_tfidf_model, f) 

In [141]:
# 2.DecisionTreeClassifier 모델 load
with open('/content/drive/MyDrive/hack/pickle/model.pkl', 'rb') as f:
  dt_tfidf_model = pickle.load(f)

- gridsearch로 진행

In [ ]:
from sklearn.model_selection import GridSearchCV

# 그리드 서치를 통해서 여러가지 파라미터를 조합할 수 있음 
param_dict = {'max_depth':[1,2,3], 'min_samples_split':[2,3]}

# 모델 훈련
grid_dt_tfidf_model = GridSearchCV(dt_tfidf_model,param_grid=param_dict, cv=3)
grid_dt_tfidf_model.fit(x_tfidf_train,y_train)


<>:10: SyntaxWarning: 'str' object is not callable; perhaps you missed a comma?
<>:10: SyntaxWarning: 'str' object is not callable; perhaps you missed a comma?
<ipython-input-92-c6e606aaf22a>:10: SyntaxWarning: 'str' object is not callable; perhaps you missed a comma?
  print(f"모델의 최고 정확도: "(grid_dt_tfidf_model.best_score_))


TypeError: ignored

In [ ]:
print(f"모델의 최고 정확도: {grid_dt_tfidf_model.best_score_}")
print(f"최적의 파라미터: {grid_dt_tfidf_model.best_params_}")

모델의 최고 정확도: 0.47462999369887027
최적의 파라미터: {'max_depth': 3, 'min_samples_split': 2}


In [ ]:
# grid search를 돌려본 결과 DecisionTree의 defalut 값이 accuarcy가 더 높은것을 볼 수 있다.
print(f"모델의 정확도: {dt_tfidf_model.score(x_tfidf_test,y_test)}")

모델의 정확도: 0.9078167595116747


In [ ]:
print(classification_report(y_test,pred_y_tfidf))

              precision    recall  f1-score   support

           0       0.89      0.89      0.89     11704
           1       0.91      0.91      0.91     10175
           2       0.92      0.94      0.93      7132
           3       0.94      0.91      0.92      4737

    accuracy                           0.91     33748
   macro avg       0.91      0.91      0.91     33748
weighted avg       0.91      0.91      0.91     33748



# 6. 결과 제출

In [138]:
import numpy as np

In [139]:
test_df['real'] = np.nan

In [145]:
test_df['real'] = dt_tfidf_model.predict(x_tfidf_test)

In [146]:
test_df

,contents,class,prepro_data,real
0,낙하물 수거 부탁드립니다. 도로 위에 낙하물이 있어요.,0,낙하 물 수거 부탁드리다 도로 위 낙하 물이 있다,0
1,창원 도시철도망 구축(수소트램)은 도시철도 노선2를 창원역에서 신창원역-남창원역-성...,1,창원 도시철도 망 구축 소 트램 도시철도 노선 2 를 창원 역 에서 신창원역 남창원...,1
2,대산면 우암리 진입도로가 너무 좁아 2차로 개설을 부탁드리고자 합니다.,0,대산면 우암 리 진입 도로 너무 좁다 2 차로 개설 을 부탁드리다 하다,3
3,성산구 건설기계 면허증 갱신기간이 궁금합니다.,0,성산구 건설 기계 면허증 갱신 기간 궁금하다,2
4,경상남도 창원시 성산구 #@주소#에서 보도에 포크레인 작업중 안전요원 없이 공사 진...,2,경상남도 창원시 성산구 주소 에서 보도 포크레인 작업중 안전 요원 없이 공사 진행 ...,2
...,...,...,...,...
33743,봉암공단 악취 발생 현장 신고와 관련하여 문의합니다,3,봉 암 공단 악취 발생 현장 신고 와 관련 하다 문의 하다,3
33744,"원동 푸르지오 공사로 소음, 분진, 매연냄새가 심각해서 무더운 여름에 창문을 열 수...",3,원동 푸르지오 공사 로 소음 분진 매 연 냄새 심각하다 무덥다 여름 창문 을 열 수...,1
33745,연지마을 입구에서 버스정류장에 가려면 반대편 인도로 건너가야 하는데 횡단보도가 없어...,1,연지 마을 입구 에서 버스정류장 가다 반대편 인도 로 건너가다 하다 횡단보도 없다 ...,0
33746,저녁만 되면 원인을 모르는 악취가 문을 열면 심하게 납니다. 문을 열 수가 없어요....,3,저녁 만 되다 원인 을 모르다 악취 문 을 열다 심하다 나다 문 을 열 수가 없다 ...,0


In [150]:
list_ = {}
for key,values in label_di.items():
  list_[values] = key

In [155]:
test_df

,contents,class,prepro_data,real,last_name
0,낙하물 수거 부탁드립니다. 도로 위에 낙하물이 있어요.,0,낙하 물 수거 부탁드리다 도로 위 낙하 물이 있다,0,NaN
1,창원 도시철도망 구축(수소트램)은 도시철도 노선2를 창원역에서 신창원역-남창원역-성...,1,창원 도시철도 망 구축 소 트램 도시철도 노선 2 를 창원 역 에서 신창원역 남창원...,1,NaN
2,대산면 우암리 진입도로가 너무 좁아 2차로 개설을 부탁드리고자 합니다.,0,대산면 우암 리 진입 도로 너무 좁다 2 차로 개설 을 부탁드리다 하다,3,NaN
3,성산구 건설기계 면허증 갱신기간이 궁금합니다.,0,성산구 건설 기계 면허증 갱신 기간 궁금하다,2,NaN
4,경상남도 창원시 성산구 #@주소#에서 보도에 포크레인 작업중 안전요원 없이 공사 진...,2,경상남도 창원시 성산구 주소 에서 보도 포크레인 작업중 안전 요원 없이 공사 진행 ...,2,NaN
...,...,...,...,...,...
33743,봉암공단 악취 발생 현장 신고와 관련하여 문의합니다,3,봉 암 공단 악취 발생 현장 신고 와 관련 하다 문의 하다,3,NaN
33744,"원동 푸르지오 공사로 소음, 분진, 매연냄새가 심각해서 무더운 여름에 창문을 열 수...",3,원동 푸르지오 공사 로 소음 분진 매 연 냄새 심각하다 무덥다 여름 창문 을 열 수...,1,NaN
33745,연지마을 입구에서 버스정류장에 가려면 반대편 인도로 건너가야 하는데 횡단보도가 없어...,1,연지 마을 입구 에서 버스정류장 가다 반대편 인도 로 건너가다 하다 횡단보도 없다 ...,0,NaN
33746,저녁만 되면 원인을 모르는 악취가 문을 열면 심하게 납니다. 문을 열 수가 없어요....,3,저녁 만 되다 원인 을 모르다 악취 문 을 열다 심하다 나다 문 을 열 수가 없다 ...,0,NaN


In [156]:
test_df["last_name"]=test_df['real'].map(list_)

In [162]:
test_df = test_df.drop(columns=['contents',"class"])

In [163]:
test_df

,prepro_data,real,last_name
0,낙하 물 수거 부탁드리다 도로 위 낙하 물이 있다,0,안전건설
1,창원 도시철도 망 구축 소 트램 도시철도 노선 2 를 창원 역 에서 신창원역 남창원...,1,교통
2,대산면 우암 리 진입 도로 너무 좁다 2 차로 개설 을 부탁드리다 하다,3,환경미화
3,성산구 건설 기계 면허증 갱신 기간 궁금하다,2,건축허가
4,경상남도 창원시 성산구 주소 에서 보도 포크레인 작업중 안전 요원 없이 공사 진행 ...,2,건축허가
...,...,...,...
33743,봉 암 공단 악취 발생 현장 신고 와 관련 하다 문의 하다,3,환경미화
33744,원동 푸르지오 공사 로 소음 분진 매 연 냄새 심각하다 무덥다 여름 창문 을 열 수...,1,교통
33745,연지 마을 입구 에서 버스정류장 가다 반대편 인도 로 건너가다 하다 횡단보도 없다 ...,0,안전건설
33746,저녁 만 되다 원인 을 모르다 악취 문 을 열다 심하다 나다 문 을 열 수가 없다 ...,0,안전건설


In [167]:
test_df.to_csv('5조.csv',encoding='CP949')